# Evaluating the Bethe model versus BEAST2

The results in this file were genererated by

1. Running `python bethe.py --subs-rate=1 -n=1001` to fit a variational posterior
2. Running BEAST 2.6.3 on `beast/M487.xml` to generate MCMC samples
3. Running [IQ-TREE 2.1.2](http://www.iqtree.org/#download) to generate a maximum likelihood tree

In [ ]:
import sys
from collections import Counter
import matplotlib.pyplot as plt
import torch
import pyro
import pyro.poutine as poutine
from pyrophylo.phylo import Phylogeny
from pyrophylo.io import stack_nexus_trees

## Load results from Bethe

In [ ]:
pyro.clear_param_store()
bethe = torch.load("results/bethe.pt")
bethe_phylo = bethe["samples"]["trees"]

In [ ]:
bethe_counts = Counter(bethe_phylo.hash_topology())
bethe_probs = {tree: count / len(bethe_phylo) for tree, count in bethe_counts.items()}
print(len(bethe_counts))
print([count / len(bethe_phylo) for tree, count in bethe_counts.most_common(10)])

In [ ]:
def plot_skyline(phylo, max_samples=100):
    phylo = phylo[:max_samples]
    num_lineages = phylo.num_lineages()
    times = phylo.times - phylo.times.max(-1, True).values
    plt.figure(figsize=(8, 4), dpi=300)
    plt.plot(times.T, num_lineages.T, 'k-', lw=1, alpha=0.5)
    plt.xlabel("time")
    plt.ylabel("number of lineages")

plot_skyline(bethe_phylo)

TODO scatterplot posterior samples of coalescent_rate

## Load results from BEAST2

In [ ]:
%%time
beast_phylo = stack_nexus_trees('beast/M487.trees')
beast_phylo = beast_phylo[501:]
assert len(beast_phylo) == 1000

In [ ]:
beast_counts = Counter(beast_phylo.hash_topology())
beast_probs = {tree: count / len(beast_phylo) for tree, count in beast_counts.items()}
print(len(beast_counts))
print([count / len(beast_phylo) for tree, count in beast_counts.most_common(10)])

In [ ]:
plot_skyline(beast_phylo)

## Load results from IQ-TREE

These results were generated by running
```sh
iqtree-2.1.2-MacOSX/bin/iqtree2 -s data/treebase/M487.nex \
  -m jc69 --date-root -1 --date-tip 0 --clock-sd 0
```

In [ ]:
iqtree_phylo = stack_nexus_trees("data/treebase/M487.nex.timetree.nex")
assert len(iqtree_phylo) == 1

In [ ]:
plot_skyline(iqtree_phylo)